In [7]:
import json
import requests
import time

import config

# Obtain the api key that was passed in from the command line
#parser = argparse.ArgumentParser(description='Sample V4')
#parser.add_argument('--api-key', type=str, default='')
#args = parser.parse_args()

API_KEY = config.API_KEY

# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports
REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited
MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited
ODDS_FORMAT = 'decimal' # decimal | american
DATE_FORMAT = 'iso' # iso | unix

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# First get a list of in-season sports
#   The sport 'key' from the response can be used to get odds in the next request
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
sports_response = requests.get('https://api.the-odds-api.com/v4/sports', params={
    'api_key': API_KEY
})

if sports_response.status_code != 200:
    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')
else:
    print('List of in season sports:', sports_response.json())


List of in season sports: [{'key': 'americanfootball_ncaaf', 'group': 'American Football', 'title': 'NCAAF', 'description': 'US College Football', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_ncaaf_championship_winner', 'group': 'American Football', 'title': 'NCAAF Championship Winner', 'description': 'US College Football Championship Winner', 'active': True, 'has_outrights': True}, {'key': 'americanfootball_nfl', 'group': 'American Football', 'title': 'NFL', 'description': 'US Football', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_nfl_super_bowl_winner', 'group': 'American Football', 'title': 'NFL Super Bowl Winner', 'description': 'Super Bowl Winner 2024/2025', 'active': True, 'has_outrights': True}, {'key': 'aussierules_afl', 'group': 'Aussie Rules', 'title': 'AFL', 'description': 'Aussie Football', 'active': True, 'has_outrights': False}, {'key': 'baseball_kbo', 'group': 'Baseball', 'title': 'KBO', 'description': 'KBO League', 'active': 

In [8]:
sports_list_json = json.loads(sports_response.text)
print(json.dumps(sports_list_json, indent=2))

[
  {
    "key": "americanfootball_ncaaf",
    "group": "American Football",
    "title": "NCAAF",
    "description": "US College Football",
    "active": true,
    "has_outrights": false
  },
  {
    "key": "americanfootball_ncaaf_championship_winner",
    "group": "American Football",
    "title": "NCAAF Championship Winner",
    "description": "US College Football Championship Winner",
    "active": true,
    "has_outrights": true
  },
  {
    "key": "americanfootball_nfl",
    "group": "American Football",
    "title": "NFL",
    "description": "US Football",
    "active": true,
    "has_outrights": false
  },
  {
    "key": "americanfootball_nfl_super_bowl_winner",
    "group": "American Football",
    "title": "NFL Super Bowl Winner",
    "description": "Super Bowl Winner 2024/2025",
    "active": true,
    "has_outrights": true
  },
  {
    "key": "aussierules_afl",
    "group": "Aussie Rules",
    "title": "AFL",
    "description": "Aussie Football",
    "active": true,
    "ha

In [9]:
count = 0
sports_for_query = []
for sport in sports_list_json:
    for attribute, value in sport.items():
        if attribute == 'key':
            if value[-6:] !='winner': #Removes futures for outright winners e.g. Superbowl, World Series
                count = count + 1
                sports_for_query.append(value)
print(f'Sport Count: {count}')

Sport Count: 60


In [10]:
sportsbook_list = [
    'wynnbet',
    'foxbet',
    'barstool'
    #'betrivers',
    #'unibet_us',
    #'superbook',
    #'fanduel',
]
SPORTSBOOKS = ','.join(sportsbook_list)

In [11]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

responses = []
#sports_for_query = ["soccer_england_league1", "soccer_england_league2"]
#sports_for_query = ["basketball_ncaab"]

for SPORT in sports_for_query:
    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds', params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT
        #'bookmakers': SPORTSBOOKS
    })

    if odds_response.status_code != 200:
        print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

    else:
        odds_json = odds_response.json()
        print('Number of events:', len(odds_json))

        responses.append(odds_json)

        # Check the usage quota
        print('Remaining requests', odds_response.headers['x-requests-remaining'])
        print('Used requests', odds_response.headers['x-requests-used'])
    time.sleep(0.1)

Number of events: 21
Remaining requests 499
Used requests 1
Number of events: 1
Remaining requests 498
Used requests 2
Number of events: 17
Remaining requests 497
Used requests 3
Number of events: 5
Remaining requests 496
Used requests 4
Number of events: 25
Remaining requests 495
Used requests 5
Number of events: 6
Remaining requests 494
Used requests 6
Number of events: 2
Remaining requests 493
Used requests 7
Number of events: 10
Remaining requests 492
Used requests 8
Number of events: 35
Remaining requests 491
Used requests 9
Number of events: 4
Remaining requests 490
Used requests 10
Number of events: 4
Remaining requests 489
Used requests 11
Number of events: 16
Remaining requests 488
Used requests 12
Number of events: 1
Remaining requests 487
Used requests 13
Number of events: 1
Remaining requests 486
Used requests 14
Number of events: 52
Remaining requests 485
Used requests 15
Number of events: 8
Remaining requests 484
Used requests 16
Number of events: 7
Remaining requests 483

In [12]:
best_odds_all_sports = {}
for response in responses:
    all_matches_best_odds = {}
    for match in response:
        best_odds = {}
        match_str = match["away_team"]+"@"+match["home_team"]
        for bookmaker in match["bookmakers"]:
            for outcome in bookmaker["markets"][0]["outcomes"]:
                if outcome["name"] not in best_odds:
                    best_odds[outcome["name"]] = [outcome["price"], bookmaker["title"]]
                elif best_odds[outcome["name"]][0] < outcome["price"]:
                    best_odds[outcome["name"]] = [outcome["price"], bookmaker["title"]]
        all_matches_best_odds[match_str] = best_odds
    best_odds_all_sports[response[0]['sport_key']] = all_matches_best_odds


In [13]:
potential_matches = {}
for sport, all_matches in best_odds_all_sports.items():
    for match_str, odds_list in all_matches.items():
        implied_probability = 0
        for outcome, odds in odds_list.items():
            implied_probability = implied_probability + (1/float(odds[0]))*100
        if implied_probability > 0 and implied_probability < 100:
            league_match = sport + "-"+match_str
            potential_matches[league_match] = odds_list
            print(sport, end= " - ")
            print(match_str,end=": \n")
            print(f'Implied Probability: {implied_probability}')
            for outcome, odds in odds_list.items():
                if odds[0] < 2:
                    american_odds = round((-100)/(odds[0]-1))
                else:
                    american_odds = round((odds[0]-1)*100)
                print(outcome, end=": ")
                print(f'{american_odds} @ {odds[1]}')
            print("")

baseball_mlb - Texas Rangers@Detroit Tigers: 
Implied Probability: 95.7042957042957
Detroit Tigers: 225 @ MyBookie.ag
Texas Rangers: -185 @ WynnBET

baseball_mlb - Pittsburgh Pirates@New York Mets: 
Implied Probability: 92.51559251559252
New York Mets: -118 @ MyBookie.ag
Pittsburgh Pirates: 160 @ DraftKings

icehockey_nhl - New York Islanders@New Jersey Devils: 
Implied Probability: 89.42307692307692
New Jersey Devils: 700 @ FanDuel
New York Islanders: -333 @ MyBookie.ag

icehockey_nhl - Buffalo Sabres@Tampa Bay Lightning: 
Implied Probability: 98.82352941176471
Buffalo Sabres: -143 @ MyBookie.ag
Tampa Bay Lightning: 150 @ Caesars

icehockey_nhl - Boston Bruins@Washington Capitals: 
Implied Probability: 98.99888765294773
Boston Bruins: 190 @ FanDuel
Washington Capitals: -182 @ MyBookie.ag

icehockey_nhl - Montréal Canadiens@Detroit Red Wings: 
Implied Probability: 80.55555555555556
Detroit Red Wings: 300 @ FanDuel
Montréal Canadiens: -125 @ MyBookie.ag

mma_mixed_martial_arts - Hayisae

In [14]:
total_match_betting_amount = 100
for match_str, odds_list in potential_matches.items():
    print(match_str)
    odds_total = 0
    for _, odds in odds_list.items():
        odds_total = odds_total + odds[0]
    for outcome, odds in odds_list.items():
        bet_amt = total_match_betting_amount/odds[0]
        net_earnings = (bet_amt*odds[0])-bet_amt

        # Convert Decimal Odds to American Odds
        if odds[0] < 2:
            american_odds = round((-100)/(odds[0]-1)) 
        else:
            american_odds = round((odds[0]-1)*100)

        print(f'Bet ${bet_amt} on [{american_odds}] {outcome}@{odds[1]} - net earnings:${net_earnings}')
    print("")

baseball_mlb-Texas Rangers@Detroit Tigers
Bet $30.76923076923077 on [225] Detroit Tigers@MyBookie.ag - net earnings:$69.23076923076923
Bet $64.93506493506493 on [-185] Texas Rangers@WynnBET - net earnings:$35.06493506493507

baseball_mlb-Pittsburgh Pirates@New York Mets
Bet $54.05405405405405 on [-118] New York Mets@MyBookie.ag - net earnings:$45.94594594594595
Bet $38.46153846153846 on [160] Pittsburgh Pirates@DraftKings - net earnings:$61.53846153846154

icehockey_nhl-New York Islanders@New Jersey Devils
Bet $12.5 on [700] New Jersey Devils@FanDuel - net earnings:$87.5
Bet $76.92307692307692 on [-333] New York Islanders@MyBookie.ag - net earnings:$23.07692307692308

icehockey_nhl-Buffalo Sabres@Tampa Bay Lightning
Bet $58.82352941176471 on [-143] Buffalo Sabres@MyBookie.ag - net earnings:$41.17647058823529
Bet $40.0 on [150] Tampa Bay Lightning@Caesars - net earnings:$60.0

icehockey_nhl-Boston Bruins@Washington Capitals
Bet $34.48275862068966 on [190] Boston Bruins@FanDuel - net ear